# Smartphone Garage Remote | 03 | Raspberry Pi Setup

## Connecting to the Raspberry Pi
One of the main benefits of using the Raspberry Pi is working with its Linux distro, Raspbian. Issuing commands to the OS can be done through a remote terminal. This can be accomplished from a working machine via SSH:
```
ssh pi@192.168.1.21
```
Additionally, Unix-based systems were designed to easily mount various filesystems into their own structure for easy access. This can be done with another fairly simple command from the working machine:
```
sshfs pi@192.168.1.21: /home/matthew/mount/pi
```

## WebIOPi
The crux of this project will be accomplished with the versitile [WebIOPi](http://webiopi.trouch.com/). WebIOPi is a web service that runs on the Raspberry Pi and allows control of the GPIO (general purpose input/output) pins on the Pi via HTTP or CoAP. Functionality can be implemented with Python.

### WebIOPi config
There are two main aspects of WebIOPi that are used for this project. The first is the config file which will setup the HTTP service, link to the Python script, and configure the pins.

#### /etc/webiopi/config
```
[GPIO]
# Initialize following GPIOs with given function and optional value
# This is used during WebIOPi start process

24 = OUT 0  

#------------------------------------------------------------------------#

[~GPIO]
# Reset following GPIOs with given function and optional value
# This is used at the end of WebIOPi stop process

24 = OUT 0

#------------------------------------------------------------------------#

[SCRIPTS]
# Load custom scripts

default = /home/pi/webiopi/default.py

#------------------------------------------------------------------------#

[HTTP]
# HTTP Server configuration

enabled     = true
port        = 8000
passwd-file = /etc/webiopi/passwd
prompt      = "WebIOPi"

#------------------------------------------------------------------------#

[COAP]
# CoAP Server configuration

enabled = false

#------------------------------------------------------------------------#

[DEVICES]

#------------------------------------------------------------------------#

[REST]

#------------------------------------------------------------------------#

[ROUTES]
# Custom REST API routes

/garage = /GPIO/24/value
```

First, during startup the config file will set our garage pin (24) to an output state with low voltage (off). At shutdown, the pin will be set to low voltage as well. 

Next, the script location is provided for the WebIOPi service. The details of this Python script will be detailed in the next section.

The HTTP service is then started on our desired port, and with our chosen username and password, as generated with WebIOPi's password generator (run with superuser privileges):
```
sudo webiopi-passwd
```
We can also create a custom REST route to our garage pin. REST (aka Representational State Transfer) is the *way* in which HTTP should be used, and part of this methodology is aided with the use of human-readable routes. But as shown below, the Python macro will take care of our access, eliminating the need for this shortcut. Direct access via the REST route was only useful during testing.

### Python script and macro
The Python scripts used by WebIOPi are Arduino-like in that they can contain *setup*, *loop*, and *destroy* functions that will be run at startup, throughout, and at the end of the system, respectably. But uniquely, macros can be defined as well. Macros are definitions that can be made within the script and then accessed via a particular REST route.

#### /home/pi/webiopi/default.py
```
import webiopi
import time
gpio = webiopi.GPIO
sleep = time.sleep

garage_pin  = 24   # set garage pin
garage_wait = 0.3  # wait time (s)

"""
Arduino-Like script functions
"""
# "setup" function called at WebIOPi startup
def setup():
    return

# "loop" function run repeatedly by WebIOPi
def loop():
    return

# "destroy" function called at WebIOPi shutdown
def destroy():
    return

"""
Macros
"""
@webiopi.macro
def garage():
    gpio.digitalWrite(garage_pin, 1)
    sleep(garage_wait)
    gpio.digitalWrite(garage_pin, 0)
    return
```
In our case, putting our garage logic inside the loop is not ideal. By defining a macro, we can operate our garage remote at any time with a direct POST command. Our macro is fairly simple; the remote is switched on, and after a short amount of time, it is switched off. The macro can be accessed via the REST route,
```
/macros/garage
```

## Port Forwarding
Within our network, the Raspberry Pi's WebIOPi service can be access through port 8000 and its local IP address. For instance, we can activate our garage macro via,
```
POST username:password@192.168.1.21:8000/macros/garage
```
However, it is not always convenient to be on our local network to activate our remote. From outside the house, obtaining a wireless connection can require waiting by the garage door for nearly 30 seconds. To fix this issue, we can fallback to the cellular data network, accessing our Raspberry Pi from outside the local network. 

To do this, we replace the local IP address of our Pi with the IP address of our house (as can be found from websites such as [this](http://canihazip.com)).  Next, we need to tell our home router to forward incoming traffic on port 8000 to the Pi. The exact method depends on the router software, but most routers allow port forwarding.